# Problem-Solving Agent with Planning, Reasoning & Domain Knowledge: illustrative example using `FinanceBench` financial-analysis dataset

This notebook illustrates the use of `OpenSSA`'s `Agent` and its planning, reasoning & domain knowledge integration capabilities to solve a problem in the financial-analysis domain.

## Setups

In [ ]:
from pprint import pprint
from IPython.display import display, Markdown

In [ ]:
import os
import sys

if cwd_is_root := ('examples' in os.listdir()):
    sys.path.append('examples')

In [ ]:
from pathlib import Path
from dotenv import load_dotenv

load_dotenv(dotenv_path=Path('examples' if cwd_is_root else '.') / '.env')

## Imports of Agent, Planning, Reasoning & Resource classes from `OpenSSA`

In [ ]:
from openssa import (Agent,
                     HTP, AutoHTPlanner,
                     OodaReasoner,
                     FileResource)

## Problem to Solve and Knowledge & Resource available for use

In [ ]:
# problem to solve
PROBLEM = 'Does AMD have a healthy liquidity profile based on FY22 Quick Ratio?'

In [ ]:
# available domain knowledge (stored as string)
from FinanceBench.data_and_knowledge import EXPERT_KNOWLEDGE as FINANCIAL_KNOWLEDGE

In [ ]:
# available informational resource: AMD's 2022 10K filing

from FinanceBench.data_and_knowledge import Doc as FinancialDoc

AMD_2022_10K = FileResource(path=FinancialDoc('AMD_2022_10K').dir_path)

display(Markdown(AMD_2022_10K.overview))

## Problem-Solving by Agent with Hierarchical Task Planning (HTP) & OODA Reasoning (OODAR)

In [ ]:
agent = Agent(planner=AutoHTPlanner(max_depth=2, max_subtasks_per_decomp=3),
              reasoner=OodaReasoner(),
              resources={AMD_2022_10K})

### Problem-Solving with Automated Dynamic Planning (default)

Without additional domain knowledge and expert inputs, the `agent` can attempt to solve the stated problem by using its Planner to decompose the problem into a 1-level-deep sub-task plan and execute that plan using its OODA Reasoner.

At any point during the OODA reasoning execution, if a confident answer cannot be established for the concerned sub-task, the `agent` would use the Planner again to decompose that sub-task 1 level further. This recursive decomposition can be done up to the `agent`'s maximum allowed planning depth.

This default solving mechanism provides a baseline that is often acceptable for domains that are popularly known/understood.

In [ ]:
solution_from_auto_plan_dynamically_executed = agent.solve(PROBLEM)

display(Markdown(solution_from_auto_plan_dynamically_executed))

## Problem-Solving with Expert-Guided Planning

One way to make the solution highly accurate and reliable is to provide the `agent` with plan from a knowledgeable expert:

In [ ]:
expert_plan = HTP.from_dict(
    {
        'task': PROBLEM,
        'sub-plans': [
            {
                'task': 'calculate Quick Ratio conservatively as (`Cash & Cash Equivalents` + `Accounts Receivable`) / Current Liabilities',
                'sub-plans': [
                    {
                        'task': 'retrieve `Cash & Cash Equivalents`, `Accounts Receivable` & `Current Liabilities` from Balance Sheet'
                    },
                ]
            },
            {
                'task': 'see whether Quick Ratio is healthy, i.e. greater than 1'
            },
        ]
    }
)

In [ ]:
expert_guided_solution = agent.solve(PROBLEM, plan=expert_plan)

display(Markdown(expert_guided_solution))

## Problem-Solving with Domain Knowledge Injection

If expert-guided solution plans are not readily available in your use case, another and sometimes lighter-weight way to achieve consistently good problem-solving outcomes is to give the `agent` access to domain-specific knowledge, so that such knowledge can be used for constructing effective solution plans for problems in the concerned domain, and for reasoning accurately during the execution process:

In [ ]:
agent_with_knowledge = Agent(planner=AutoHTPlanner(max_depth=2, max_subtasks_per_decomp=3),
                             reasoner=OodaReasoner(),
                             knowledge={FINANCIAL_KNOWLEDGE},
                             resources={AMD_2022_10K})

In [ ]:
solution_from_auto_plan_dynamically_executed_with_knowledge = agent_with_knowledge.solve(PROBLEM, dynamic=False)

display(Markdown(solution_from_auto_plan_dynamically_executed_with_knowledge))